Initial setting.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Define the sampling patch image information and batch size.

In [ ]:
batch_size = 2
patch_size = 256
image_size = patch_size

Import the libraries needed.

In [ ]:
from __future__ import print_function
import os,sys,random,cv2,re
import tensorflow as tf
import tifffile
from tensorflow import keras
#from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from tensorflow.keras.losses import binary_crossentropy


from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import shutil
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import LearningRateScheduler

from zipfile import ZipFile

Sampling patch for all the input image(output 270 pieces for both labels and images) 

In [ ]:
imageDir = "../input/images/"
labelDir = "../input/labels/"
def patchDiv(imgPath,labelPath,patchSize=256):
    #loop through image and label
    images = os.listdir(imageDir)
    labels = os.listdir(labelDir)
    imagePatch = []
    labelPatch = []
    count = 1
    for i in images:
#         print(imgPath + i)
        imageNumber = i.split(".jpg")[0][-2:]
        imageLabel = "/train-labels" + imageNumber + ".jpg"
        
        img = cv2.imread(imgPath + i,0)
        label = cv2.imread(labelPath + imageLabel,0)
        img = np.expand_dims(img,axis=2)
        label = np.expand_dims(label,axis=2)
        cutW = img.shape[0]//patchSize
        cutH = img.shape[1]//patchSize
        #divide image into patch
        for w in range(0, (cutW * 10 // 2) + 5, 5):
            for h in range(0, (cutH * 10 // 2) + 5, 5):
                imagePatch.append(img[int((w/10)*patchSize):int(((w/10)+1)*patchSize), int((h/10)*patchSize):int(((h/10)+1)*patchSize)])
                labelPatch.append(label[int((w/10)*patchSize):int(((w/10)+1)*patchSize), int((h/10)*patchSize):int(((h/10)+1)*patchSize)])
    return imagePatch,labelPatch
imgP,labeP = patchDiv(imageDir,labelDir,patchSize=patch_size)

Print out the first all of images and label that belong to first images and label.(For checking sampling patch is correct)

In [ ]:
# plt.figure(figsize=(100,100))
# images = imgP[162:171]#os.listdir(imageDir)
# labels = labeP[162:171]#os.listdir(labelDir)

# for i in range(len(images)):
#     plt.subplot(len(images),len(labels),i+1)
#     plt.imshow(images[i][:,:,0],'gray')
#     plt.subplot(len(images),len(labels),i+len(images)+1)
#     plt.imshow(labels[i][:,:,0],'gray')
# plt.show

Split the data into train and validation.(Inorder to using validation when training the data)

In [ ]:
trainX,validX,trainY,validY = train_test_split(imgP,labeP,train_size=0.8, random_state = 2) 

The data augmentation part.(All the patch that split by sampling patch)

In [ ]:
#1. 数据增强 Data augmentation的参数

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    stacked = []
    for i,item in enumerate(npyfile):
        img = item[:,:,0].astype('float32')
        img = np.expand_dims(img,axis=0)
        if(len(stacked)==0):
            stacked = img
        else:
            stacked = np.vstack((stacked,img))
    tifffile.imsave(save_path + '/allPredict.tif', stacked)

def adjustData(img,mask,flag_multi_class,num_class):
    if(np.max(img) > 1):
        #divided by 255 to convert to probability image
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='reflect')

def trainGenerator(batch_size,trainImage,trainLable,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow(
        x = np.array(trainImage),
        batch_size = batch_size,
        shuffle = True,
        seed = seed,
        save_to_dir = save_to_dir 
    )
    mask_generator = mask_datagen.flow(
        x = np.array(trainLable),
        batch_size = batch_size,
        shuffle = True,
        seed = seed,
        save_to_dir = save_to_dir 
    )

    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
        
myGene = trainGenerator(batch_size,trainX,trainY,data_gen_args,save_to_dir = None)
myValid = trainGenerator(batch_size,validX,validY,data_gen_args,save_to_dir = None)

Unet model.

In [ ]:
def down_block(x,filters,kernel_size=(3,3),padding='same',strides=1):
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(x)
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(c)
    p=keras.layers.MaxPool2D((2,2),(2,2))(c)
    return c,p
def up_block(x,skip,filters,kernel_size=(3,3),padding='same',strides=1):
    us=keras.layers.UpSampling2D((2,2))(x)
    concat=keras.layers.concatenate([us,skip],axis=3)
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(concat)
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(c)
    return c
def bottleNeck(x,filters,kernel_size=(3,3),padding='same',strides=1):
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(x)
    c=keras.layers.Conv2D(filters,kernel_size,padding=padding,strides=strides,activation='relu')(c)
    return c
def Unet():
    f = [64,128,256,512,1024]#feature maps, number of filters
    inputs=keras.layers.Input((image_size,image_size,1))#input to the model
    p0 = inputs
    c1,p1 = down_block(p0,f[0])#1->64
    c2,p2 = down_block(p1,f[1])#64->128
    c3,p3 = down_block(p2,f[2])#128->256
    c4,p4 = down_block(p3,f[3])#256->512
    drop1 = keras.layers.Dropout(0.5)(p4)
    
    bn = bottleNeck(drop1,f[4])#512->1024
    drop2 = keras.layers.Dropout(0.5)(bn)

    u1 = up_block(drop2,c4,f[3])#1024->512
    u2 = up_block(u1,c3,f[2])#512->256
    u3 = up_block(u2,c2,f[1])#256->128
    u4 = up_block(u3,c1,f[0])#128->64
    
    outputs_up = keras.layers.Conv2D(2,(3,3),padding="same",activation="relu")(u4)#64->2
    outputs = keras.layers.Conv2D(1,(1,1),padding="same",activation="sigmoid")(outputs_up)#2->1
    print("output",outputs.shape)
    model = keras.models.Model(inputs,outputs)
    return model

In [ ]:
# def Unet(pretrained_weights = None,input_size = (256, 256, 1)):
#     inputs = keras.layers.Input(input_size)
#     conv1 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
#     conv1 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
#     pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
#     conv2 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
#     pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
#     conv3 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
#     pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
#     conv4 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
#     drop4 = keras.layers.Dropout(0.5)(conv4)
#     pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

#     conv5 = keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
#     conv5 = keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
#     drop5 = keras.layers.Dropout(0.5)(conv5)

#     up6 = keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
#     merge6 = keras.layers.concatenate([drop4,up6], axis = 3)
#     conv6 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
#     conv6 = keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

#     up7 = keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#     merge7 = keras.layers.concatenate([conv3,up7], axis = 3)
#     conv7 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
#     conv7 = keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

#     up8 = keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
#     merge8 = keras.layers.concatenate([conv2,up8], axis = 3)
#     conv8 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
#     conv8 = keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

#     up9 = keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
#     merge9 = keras.layers.concatenate([conv1,up9], axis = 3)
#     conv9 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
#     conv9 = keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv9 = keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#     conv10 = keras.layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)

#     model = keras.models.Model(inputs,conv10)

#     model.compile(optimizer = Adam(lr = 1e-4), loss = bce_dice_loss, metrics = ['accuracy', dice_coef])
#     #model.compile(optimizer = Adam(lr = 1e-4), loss = focal_loss(gamma=2,alpha=0.6), metrics = ['accuracy', dice_coef])
#     #model.compile(optimizer = Adam(lr = 1e-4), loss = focal_dice_loss, metrics = ['accuracy', dice_coef])
    
#     if(pretrained_weights):
#         model.load_weights(pretrained_weights)

#     return model

Loss function(Dice coefficeient and binary cross Dice coefficeient)

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth = 1):
    return 1- dice_coef(y_true, y_pred, smooth)

def bce_dice_loss(y_true, y_pred, weight = 1, smooth = 1):
    '''
    bce + w*dice loss
    '''
    return binary_crossentropy(y_true, y_pred) + weight*dice_loss(y_true, y_pred)

Compile the model and training.

In [ ]:
model = Unet()
opt = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=opt, loss=bce_dice_loss, metrics=["acc"])#metrics=[dsc]
trainStep = len(trainX)//batch_size
validStep = len(validX)//batch_size
print(trainStep)
print(validStep)
history = model.fit_generator(myGene,steps_per_epoch=trainStep,epochs=80,validation_data=myValid,validation_steps=validStep)

Check out for the train loss and accuracy.

In [ ]:
def show_final_history(history):
    fig= plt.figure(figsize=(10,10))
    plt.title('Loss')
    plt.plot(history.epoch, history.history["loss"], label="Train loss")
    plt.plot(history.epoch, history.history["val_loss"], label="Validation loss")
    plt.legend()
    fig= plt.figure(figsize=(10,10))
    plt.title('Acc')
    plt.plot(history.epoch, history.history["acc"], label="Train acc")
    plt.plot(history.epoch, history.history["val_acc"], label="Validation acc")
    plt.legend()
    fig= plt.figure(figsize=(10,10))
show_final_history(history)
print("Validation Accuracy: " + str(history.history['val_acc'][-1:]))

Generate the testing data.

In [ ]:
TestList = ["../input/images/train-volume0%d.jpg"%i if(i<10) else "../input/images/train-volume%d.jpg"%i for i in range(30)]

In [ ]:
def TestDataGenerator(imageList,patchSize=256):
    for img in imageList:
        x = cv2.imread(img,0)#256,256
        x = np.expand_dims(x,axis=2)#256,256,1
        x = np.expand_dims(x,axis=0)#1,256,256,1
        num = x.shape[1]//patchSize # 2
        
        for i in range(0, (num * 10 // 2) + 5, 5):
            for j in range(0, (num * 10 // 2) + 5, 5):
                yield x[:,int((i/10)*patchSize):int(((i/10)+1)*patchSize),int((j/10)*patchSize):int(((j/10)+1)*patchSize)]

In [ ]:
testGene = TestDataGenerator(TestList)

The part that merge all the pieces that split by sampling patch into a completed image.

In [ ]:
#merge the result 
def mergeResult(imageList,originalImageShape=512,patchSize=256):
    #index: number of elements in the input list for one image
    cut = (originalImageShape//patchSize) + 1 #512 / 256 + 1 = 3
    index = cut * cut
    count = 0
    i = 0
    print(len(imageList))
    mergedResult = np.zeros((len(imageList)//index, ) + (originalImageShape,originalImageShape,1))
    while(count + index <= len(imageList)):#(9,18,27...) <= 270
        vstacked = []
        temp = imageList[count:count + index]#[0:9]
        flag = 0
        hstacked_list = []
        for h in range(cut,index + 1, 3):#(2-4)
            tmp = []
            tmp2 = []
            for r in range(flag,flag + cut):#(0-2),(2-4)
                tmp.append(temp[r][:,:128,:])
                tmp.append(temp[r][:,128:256,:])
            tmp2.append(tmp[0])
            tmp2.append((tmp[1] + tmp[2]) / 2)
            tmp2.append((tmp[3] + tmp[4]) / 2)
            tmp2.append(tmp[5])
            
            for k in range(len(tmp2)):
                hstacked = tmp2[k] if(k == 0) else np.hstack((hstacked,tmp2[k]))
            hstacked_list.append(hstacked)
            flag = h
            
        vtmp = []
        vtmp2 = []
        for j in range(len(hstacked_list)):
            vtmp.append(hstacked_list[j][:128,:,:])
            vtmp.append(hstacked_list[j][128:256,:,:])
        vtmp2.append(vtmp[0])
        vtmp2.append((vtmp[1] + vtmp[2]) / 2)
        vtmp2.append((vtmp[3] + vtmp[4]) / 2)
        vtmp2.append(vtmp[5])
        for l in range(len(vtmp2)):
            vstacked = vtmp2[l] if (len(vstacked)==0) else np.vstack((vstacked,vtmp2[l]))
        count += index
        mergedResult[i] = vstacked
        i += 1 
    return mergedResult

Output and save the result.

In [ ]:
myoutputDir = "./output"
if not os.path.exists(myoutputDir):
    os.makedirs(myoutputDir)
    
result = model.predict_generator(testGene,steps=30*9,verbose=0)
print("output has %d images"%len(result))
print(result[0].shape)
result = mergeResult(result)
print("final output after merging has %d images"%len(result))
plt.imshow(result[0][:,:,0],cmap="gray")
plt.show
saveResult(myoutputDir,result)
with ZipFile("./testOut.zip","w") as zip:
    for file in os.listdir(myoutputDir):
        zip.write(myoutputDir + "/" + file)
        
print(list(zip.infolist()))